In [ ]:
import os

import numpy as np
import cv2
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
LETTERS = list("АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")
SIZE = 68

In [ ]:
import zipfile    	
fantasy_zip = zipfile.ZipFile('/content/drive/MyDrive/dataset.zip')
fantasy_zip.extractall('/content')
fantasy_zip.close()

In [ ]:
model = Sequential()
model.add(Convolution2D(filters=64, kernel_SIZE=(5, 5), padding='valid', input_shape=(SIZE, SIZE, 1), activation='relu'))
model.add(Convolution2D(filters=128, kernel_SIZE=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_SIZE=(3, 3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(LETTERS), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
# Change color space to gray, normalize value.
def normalize(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) / 255.0
    image = cv2.reSIZE(image, (SIZE,SIZE), cv2.INTER_AREA)
    return image


x_train = []
x_test = []
y_train = []
y_test = []

path = "/content/test (копия)"
for folder in os.listdir(path):
    x_temp = []
    y_temp = []
    
    for image in os.listdir(f'{path}/{folder}'):
        x_temp.append(normalize(cv2.imread(f'{path}/{folder}/{image}')))
        y_temp.append(LETTERS.index(folder))
    
    # Devide dataset to train and test samples.
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(x_temp, y_temp, stratify = y_temp, test_size = 0.2)
    
    x_train.extend(x_train_temp)
    x_test.extend(x_test_temp)
    y_train.extend(y_train_temp)
    y_test.extend(y_test_temp)

# Shape data to input form of model.
x_train = np.array(x_train)
x_train = np.reshape(x_train, (x_train.shape[0], SIZE, SIZE, 1))
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], SIZE, SIZE, 1))
y_train = np.array(y_train)
y_test = np.array(y_test)

x_train_cat = keras.utils.to_categorical(y_train, len(LETTERS))
y_test_cat = keras.utils.to_categorical(y_test, len(LETTERS))

# Save formed dataset to file (to retrain the model later).
np.savez_compressed('/content/drive/MyDrive/prepared_dataset.npz',
                    x_train=x_train, x_train_cat=x_train_cat, x_test=x_test, y_test_cat=y_test_cat)

In [ ]:
dataset = np.load('/content/drive/MyDrive/prepared_dataset.npz')
x_train = dataset['x_train']
x_train_cat = dataset['x_train_cat']

In [ ]:
model.fit(x_train, x_train_cat, batch_SIZE=128, epochs=100)

In [ ]:
x_test = dataset['x_test']
y_test_cat = dataset['y_test_cat']

results = model.evaluate(x_test, y_test_cat, batch_SIZE=128)
print('test loss, test acc:', results)

In [ ]:
model.save('/content/drive/MyDrive/exp_model.h5')